OLD NOTEBOOK: SEE osm_pbf_power_data_extractor.py which does everything.

In [1]:
import os, sys, time

# IMPORTANT: RUN SCRIPT FROM THIS SCRIPTS DIRECTORY i.e data_exploration/ TODO: make more robust
##os.chdir(os.path.dirname(os.path.abspath(__file__)))
sys.path.append("../../scripts")
from osm_data_config import AFRICA_CC


import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString
import geoplot
import matplotlib.pyplot as plt
from osm_data_config import AFRICA_CC
from osm_pbf_power_data_extractor import convert_pd_to_gdf_lines, convert_pd_to_gdf

import logging

logger = logging.getLogger(__name__)

# SUBSTATIONS

In [2]:
# ----------- SUBSTATIONS -----------
# Load uncleaned data
df_all_substations = pd.read_csv(os.getcwd() + "/data/africa_all_substations.csv")
# Clean
df_all_substations = df_all_substations.reset_index(drop=True)
df_all_substations = df_all_substations.dropna(
    subset=["tags.voltage"]
)  # Drop any substations with Voltage = N/A

# ~ Generate Files

# CSV
# outputfile_partial = os.path.join(os.getcwd(),'data','africa_all'+'_substations'+'_cleaned.')
# df_all_substations.to_csv(outputfile_partial + 'csv') # Generate CSV

# GeoJSON
##gdf_substations = convert_pd_to_gdf(df_all_substations)
##gdf_substations.to_file(outputfile_partial+'geojson', driver="GeoJSON")  # Generate GeoJson


display(df_all_substations)

,Unnamed: 0,id,lonlat,tags.power,tags.substation,tags.voltage,Type,Country
4,95,217933343.0,"[5.259872990608272, 31.93945777215369]",substation,NaN,220000;66000,Way,algeria
6,111,533163474.0,"[6.982630706902727, 34.50483341084844]",substation,transmission,220000,Way,algeria
7,117,226886714.0,"[5.372428775748285, 35.34846375188343]",substation,transmission,220000;66000,Way,algeria
9,123,107574175.0,"[3.384622541387728, 36.72704942077627]",substation,transmission,220000,Way,algeria
10,129,388340182.0,"[6.9876692277391745, 35.0307310287779]",substation,NaN,66000,Way,algeria
...,...,...,...,...,...,...,...,...
2078,148,563464603.0,"[28.445422379846164, -15.18135199278004]",substation,NaN,330000,Way,zambia
2091,111,642251151.0,"[29.224362618439557, -19.689381166027218]",substation,transmission,132000,Way,zimbabwe
2094,123,651771592.0,"[31.606930427414476, -18.15115641531688]",substation,transmission,132000,Way,zimbabwe
2095,129,525189536.0,"[30.162416456820235, -19.345625582780112]",substation,transmission,132000,Way,zimbabwe


# LINES 

In [31]:
# ----------- LINES -----------

# Clean
# TODO: FIX Voltage Filter
# Some transmission lines carry multiple voltages, having voltage_V = 10000;20000  (two lines)
# The following code keeps only the first information before the semicolon..
# Needs to be corrected in future, creating two lines with the same bus ID.

# Load unclean data
df_all_lines = pd.read_csv(os.getcwd() + "/data/africa_all_lines.csv")


# Clean data
df_all_lines = df_all_lines.reset_index(drop=True)
df_all_lines = df_all_lines.drop(columns="Unnamed: 0")
df_all_lines = df_all_lines.drop(columns="lonlat")
df_all_lines = df_all_lines.dropna(
    subset=["tags.voltage"]
)  # Drop any lines with Voltage = N/A
df_all_lines = df_all_lines.rename(columns={"tags.voltage": "voltage_V"})
df_all_lines["voltage_V"] = (
    df_all_lines["voltage_V"].str.split("*").str[0]
)  # just keeps the
df_all_lines["voltage_V"] = df_all_lines["voltage_V"]  # .str.split(';').str[0]
df_all_lines["voltage_V"] = (
    df_all_lines["voltage_V"]
    .apply(lambda x: pd.to_numeric(x, errors="coerce"))
    .dropna()
)  ## if cell can't converted to float -> drop
df_all_lines = df_all_lines[df_all_lines.voltage_V > 10000]

# Create Final dataframe
# line_id,bus0,bus1,voltage,circuits,length,underground,under_construction,tags,geometry
df_all_lines.rename(columns={"tags.voltage": "voltage_V"})


## Generate Files
## CSV
# outputfile_partial = os.path.join(os.getcwd(), 'data', 'africa_all'+'_lines'+'_cleaned.')
# df_all_lines.to_csv(outputfile_partial + 'csv')  # Generate CSV
## GeoJSON
# gdf_lines = convert_pd_to_gdf_lines(df_all_lines, simplified=True)
# gdf_lines.to_file(outputfile_partial+'geojson',
#             driver="GeoJSON")  # Generate GeoJson

display(df_all_lines)

,id,tags.power,tags.cables,voltage_V,tags.circuits,tags.frequency,Type,Country
1,262871070,line,3.0,66000.0,NaN,NaN,Way,algeria
2,655094635,line,6.0,220000.0,NaN,50.0,Way,algeria
3,378839793,line,3.0,60000.0,NaN,50.0,Way,algeria
4,382460532,line,3.0,400000.0,NaN,NaN,Way,algeria
5,380786702,line,6.0,400000.0,2.0,50.0,Way,algeria
...,...,...,...,...,...,...,...,...
21420,651546006,line,3.0,330000.0,NaN,50.0,Way,zimbabwe
21421,651760976,line,3.0,132000.0,NaN,50.0,Way,zimbabwe
21422,200790530,line,3.0,330000.0,NaN,NaN,Way,zimbabwe
21423,651760977,line,3.0,132000.0,NaN,50.0,Way,zimbabwe


In [30]:
X = df_all_lines.voltage_V

print("Unique values", X.unique())
print()
print("Datatype:", X.dtype)

Unique values [ 66000. 220000.  60000. 400000.  22000.  90000.  30000. 150000. 225000.
 132000. 350000. 110000. 330000. 161000.  70000. 120000. 500000.  11000.
  63000. 230000.  45000. 115000.  88000.  69000.  25000. 138000.  33000.
  20000.  22500. 533000. 275000.  19000. 765000.  44000. 380000.  15000.]

Datatype: float64


# GENERATORS

In [4]:
# ----------- Generator -----------

# Load uncleaned data
df_all_generators = pd.read_csv(os.getcwd() + "/data/africa_all_generators.csv")

# Clean data
df_all_generators = df_all_generators.reset_index(drop=True)
df_all_generators = df_all_generators[
    df_all_generators["tags.generator:output:electricity"]
    .astype(str)
    .str.contains("MW")
]  # removes boolean
df_all_generators["tags.generator:output:electricity"] = (
    df_all_generators["tags.generator:output:electricity"]
    .str.extract("(\d+)")
    .astype(float)
)
df_all_generators = df_all_generators.rename(
    columns={"tags.generator:output:electricity": "power_output_MW"}
)


## Generate Files

# CSV
# outputfile_partial = os.path.join(os.getcwd(),'data','africa_all'+'_generators'+'_cleaned.')
# df_all_generators.to_csv(outputfile_partial + 'csv') # Generate CSV

# GeoJSON
# gdf_generators = convert_pd_to_gdf(df_all_generators)
# gdf_generators.to_file(outputfile_partial+'geojson', driver="GeoJSON")  # Generate GeoJson


display(df_all_generators)

,Unnamed: 0,id,lonlat,Area,tags.power,tags.generator:type,tags.generator:method,tags.generator:source,power_output_MW,Type,Country
0,0,7.781384e+09,"[6.023165900000033, 31.760752899999908]",NaN,generator,gas_turbine,combustion,gas,123.0,Node,algeria
1,1,7.781385e+09,"[3.0654618000000364, 36.76114909999989]",NaN,generator,gas_turbine,combustion,gas,36.0,Node,algeria
3,3,6.424707e+09,"[6.0504717999999915, 31.788152899999794]",NaN,generator,gas_turbine,combustion,gas,24.0,Node,algeria
4,4,7.781385e+09,"[5.258395700000035, 31.93974219999988]",NaN,generator,gas_turbine,combustion,gas,24.0,Node,algeria
5,5,7.781321e+09,"[4.880102300000034, 36.65123149999992]",NaN,generator,gas_turbine,combustion,gas,23.0,Node,algeria
...,...,...,...,...,...,...,...,...,...,...,...
11902,17,7.979052e+09,"[28.76224349999994, -16.519499199999984]",NaN,generator,NaN,NaN,hydro,180.0,Node,zimbabwe
11903,18,7.979052e+09,"[28.76234359999994, -16.519360299999985]",NaN,generator,NaN,NaN,hydro,180.0,Node,zimbabwe
11904,19,7.979052e+09,"[28.76439309999994, -16.52185269999999]",NaN,generator,NaN,NaN,hydro,125.0,Node,zimbabwe
11905,20,7.979052e+09,"[28.76212659999994, -16.519661299999985]",NaN,generator,NaN,NaN,hydro,180.0,Node,zimbabwe
